# Similarities & Distances

## 0. Поиск ключевых слов по 4-м тематикам
- спорт
- наука
- шоппинг
- новости

На просторах интернета есть куча сайтов с базами данных ключевых слов по определённой тематике, но, of course, почти все они платные.... :sob:

Сначала я хотела составить базу слов, основываясь на данных с сервисов [Google Trends](https://trends.google.ru/trends/) (у этого сервиса удобнее интерфейс, и есть возможность скачать .csv с related запросами) и [Yandex Wordstat](https://wordstat.yandex.ru/). Но они предоставляют ограниченное кол-во слов, и в большинстве случаев предоставляемые слова (по моему мнению) не очень хорошо описывают данную тематику.

Поэтому я использовала сервис [kartaslov.ru](https://kartaslov.ru/), на этом сайте можно получить синонимичные слова, слова, которые ассоциируются с данным топиком - это является более характеризующим/описательным для определения принадлежности какого-то текста к данной тематике. (Найденные слова также подверглись манипуляциям из 1 пункта, удалены дубликаты)

Слова находятся в директории data/topics.


## 1. Prepocessing

Предобработаем выбранный текст, находящийся в файле data/oreologia.txt.
Текст взят с этого занимательного поста на хабре - [ссылка](https://habr.com/ru/company/ua-hosting/blog/663380/)


In [1]:
import nltk
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/renett/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/renett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Токенизируем (разбиваем текст на единицы - слова, знаки препинания)

In [3]:
file_name = "data/oreologia.txt"

with open(file_name, 'r') as reader:
    text = reader.readlines()

tokens = []
for line in text:
    tokens += word_tokenize(line)

# print(tokens)
print(len(tokens))

3577


Убираем стоп-слова и символы пунктуации:

(тут, по-хорошему, нужно было бы ещё удалить и символы латинского алфавита - они использовались для описания формул в данном тексте)

In [5]:
# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords
from string import punctuation

noise = stopwords.words('russian') + list(punctuation)
noise.append("т.")
noise.append("д")
noise.append("—")
noise.append("«")
noise.append("»")

print(noise)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [6]:
filtered_tokens = [x for x in tokens if x.lower() not in noise]

print('Tokens after preprocessing:', len(filtered_tokens))
# print(filtered_tokens)

Tokens after preprocessing: 2186


А теперь получим начальные формы слов - лемматизируем, используя морфологический анализатор MorphAnalyzer из pymorphy2.

In [7]:
from pymorphy2 import MorphAnalyzer

pymorphy2_analyzer = MorphAnalyzer()

final_tokens = []
for token in filtered_tokens:
    lem = pymorphy2_analyzer.parse(token)
    final_tokens.append(lem[0].normal_form)
# print(final_tokens)

## 2. Считываем слова из топиков: news_tokens, science_tokens, shopping_tokens and sport_tokens

In [93]:

with open("data/topics/news.txt", 'r') as reader:
    text = reader.readlines()

news_tokens = []
for line in text:
    news_tokens += word_tokenize(line)

with open("data/topics/science.txt", 'r') as reader:
    text = reader.readlines()

science_tokens = []
for line in text:
    science_tokens += word_tokenize(line)

with open("data/topics/shopping.txt", 'r') as reader:
    text = reader.readlines()

shopping_tokens = []
for line in text:
    shopping_tokens += word_tokenize(line)

with open("data/topics/sport.txt", 'r') as reader:
    text = reader.readlines()

sport_tokens = []
for line in text:
    sport_tokens += word_tokenize(line)

topics_titles = 'news', 'science', 'shopping', 'sport'
topics_tokens = news_tokens, science_tokens, shopping_tokens, sport_tokens



## 3. Выссчитываем схожесть текста к топикам по метрике Жаккарда.


In [94]:
def jaccard_similarity(one, another):
    intersection = len(set.intersection(*[set(one), set(another)]))
    union = len(set.union(*[set(one), set(another)]))
    return intersection / float(union)

In [111]:
jaccard_similarities = []

for topic in topics_tokens:
    jaccard_similarities.append(jaccard_similarity(final_tokens, topic))

for i in range(0, 4):
    print('Jaccard similarity with topic "%s" = %.3f' % (topics_titles[i], jaccard_similarities[i]))

Jaccard similarity with topic "news" = 0.023
Jaccard similarity with topic "science" = 0.064
Jaccard similarity with topic "shopping" = 0.015
Jaccard similarity with topic "sport" = 0.029




## 4. Выссчитываем схожесть текста к топикам по метрике Косинуса.


In [98]:
from math import sqrt
import numpy


def squared_sum(x):
    return round(sqrt(sum([a * a for a in x])), 5)


def count_occurrences(elem, elems):
    count = 0
    elems = list(elems)
    for k in range(0, len(elems)):
        if elem == elems[k]:
            count += 1
    return count


def cosine_similarity(one, another):
    all_elems = set(list(one) + list(another))

    x = numpy.zeros(len(all_elems), dtype=int)
    y = numpy.zeros(len(all_elems), dtype=int)

    for i in range(0, len(all_elems)):
        elems_list = list(all_elems)
        x[i] = count_occurrences(elems_list[i], one)
        y[i] = count_occurrences(elems_list[i], another)

    numerator = sum(a * b for a, b in zip(x, y))
    denominator = squared_sum(x) * squared_sum(y)
    return round(numerator / float(denominator), 3)

In [110]:
cosine_similarities = []

for topic in topics_tokens:
    cosine_similarities.append(cosine_similarity(final_tokens, topic))

for i in range(0, 4):
    print('Cosine similarity with topic "%s" = %.3f' % (topics_titles[i], cosine_similarities[i]))

Cosine similarity with topic "news" = 0.029
Cosine similarity with topic "science" = 0.088
Cosine similarity with topic "shopping" = 0.021
Cosine similarity with topic "sport" = 0.040





## 5. Результат:



In [112]:
print('Text is from file', file_name, '\n')

for i in range(0, 4):
    print('Jaccard similarity with topic "%s" = %.3f' % (topics_titles[i], jaccard_similarities[i]))

print('\n')

for i in range(0, 4):
    print('Cosine similarity with topic "%s" = %.3f' % (topics_titles[i], cosine_similarities[i]))

Text is from file data/oreologia.txt 

Jaccard similarity with topic "news" = 0.023
Jaccard similarity with topic "science" = 0.064
Jaccard similarity with topic "shopping" = 0.015
Jaccard similarity with topic "sport" = 0.029


Cosine similarity with topic "news" = 0.029
Cosine similarity with topic "science" = 0.088
Cosine similarity with topic "shopping" = 0.021
Cosine similarity with topic "sport" = 0.040


--------------------


Видим, что по обеим метрикам текст с [хабра про исследование печенья Орео](https://habr.com/ru/company/ua-hosting/blog/663380/) ближе по содержанию к теме "НАУКА" (результат совпал с ожидаемым).


## Рейтинг тем, к которым ближе всего данный текст, по метрике косинуса:
1. наука - 0.064
2. спорт - 0.029
3. новости - 0.023
4. шоппинг - 0.015

## Рейтинг тем, к которым ближе всего данный текст, по метрике косинуса:
1. наука - 0.088
2. спорт - 0.040
3. новости - 0.020
4. шоппинг - 0.021

Рейтинги совпадают.
